In [ ]:
import numpy as np
from sklearn.metrics import confusion_matrix, accuracy_score
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import label_binarize
from modules.KNN import *
from modules.stats import * 
from modules.PCA import * 
from modules.LDA import * 
from modules.read_data import *
from modules.ROC import *

In [ ]:
path = '../../Assets/Image Dataset'
data_class = ['coast', 'forest', 'highway', 'mountain', 'opencountry']

C = [] #36*23
D = []

for i in data_class:
	t = read_ID(path, i, 'train').flatten().reshape(-1, 36*23)
	d = read_ID(path, i, 'dev').flatten().reshape(-1, 36*23)
	C.append((t - t.min(axis = 0))/(t.max(axis=0) - t.min(axis=0)))
	D.append((d - t.min(axis = 0))/(t.max(axis=0) - t.min(axis=0)))

In [ ]:
C_y = []
D_y = []

for i in range(5):
	C_y.append(i * np.ones(C[i].shape[0]))
	D_y.append(i * np.ones(D[i].shape[0]))

In [ ]:
X =  np.concatenate(C, axis=0)
X_d = np.concatenate(D, axis=0)
Y = np.concatenate(C_y, axis=0)
Y_d = np.concatenate(D_y, axis=0)

In [ ]:
pca = False
lda = False
lda = ~pca & lda #if pca is true then lda is false

In [ ]:
if pca:
	pc = 25
	μ, Σ = stats(X)
	eigval, Q = PCA(Σ, pc)
	X = X @ Q
	X_d = X_d @ Q
if lda:
	pc = 25
	μ, Σ = stats(X)
	μk = []
	for c in range(5):
		μk.append(stats(X[Y == c])[0])
	μk = np.array(μk)
	v = LDA(X,Y,μ,μk,pc)
	X = np.abs(X @ v)
	X_d = np.abs(X_d @ v)

In [ ]:
if pca or lda:	
	D_new = []
	nd=0
	for i in range(5):
		d = X_d[nd:nd+D[i].shape[0]]
		D_new.append(d)
		nd += D[i].shape[0]
	D = D_new

In [ ]:
##KNN implementation
total_train = X; dev_data = D
kn = 15
prediction = []
P = []
for c in range(5):
	p = np.zeros(dev_data[c].shape[0])
	a = np.zeros((dev_data[c].shape[0], 5))
	for i in range(dev_data[c].shape[0]):
		p[i] = KNN(dev_data[c][i], total_train, Y, kn)[0]
		A = KNN(dev_data[c][i], total_train, Y, kn)[1]
		a[i] = np.array([(A==0).sum()/kn, (A==1).sum()/kn, (A==2).sum()/kn, (A==3).sum()/kn, (A==4).sum()/kn])
	prediction.append(p)
	P.append(a)
P = np.concatenate(P)

In [ ]:
y_pred = np.concatenate(prediction)

In [ ]:
cm = confusion_matrix(Y_d, y_pred)
accuracy = accuracy_score(Y_d,y_pred)
ax = sns.heatmap(cm, annot = True, cmap ='plasma',linecolor ='black', linewidths = 1, fmt = '.0f', xticklabels = data_class, yticklabels = data_class)
ax.set_title(f'Accuracy = {accuracy}')

In [ ]:
TPR = 0; FPR = 0
for c in range(5):
	tpr, fpr = ROC(label_binarize(Y_d, classes=[0,1,2,3,4])[:,c], P[:,c])
	TPR += tpr/5
	FPR += fpr/5
plt.plot(FPR, TPR)
plt.xlabel('FPR')
plt.ylabel('TPR')
plt.title("ROC")